<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/NasNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 15.7 MB/s 
     |████████████████████████████████| 182 kB 75.0 MB/s 
     |████████████████████████████████| 162 kB 68.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 162 kB 69.6 MB/s 
     |████████████████████████████████| 158 kB 73.7 MB/s 
     |████████████████████████████████| 157 kB 75.5 MB/s 
     |████████████████████████████████| 157 kB 72.6 MB/s 
     |████████████████████████████████| 157 kB 74.2 MB/s 
     |████████████████████████████████| 157 kB 75.9 MB/s 
     |████████████████████████████████| 157 kB 75.8 MB/s 
     |████████████████████████████████| 157 kB 71.9 MB/s 
     |████████████████████████████████| 157 kB 56.7 MB/s 
     |████████████████████████████████| 156 kB 70.9 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [3]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [4]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [5]:
nasnetMobile = tf.keras.applications.NASNetMobile(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_nasnetMobile=Sequential([
  #data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  nasnetMobile,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  #layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
nasnetMobile.trainable=False
model_nasnetMobile.summary()

19993432/19993432 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 flatten (Flatten)           (None, 51744)             0         
                                                                 
 dense (Dense)               (None, 32)                1655840   
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 5,925,721
Trainable params: 1,656,005
Non-trainable params: 4,269,716
_____________________________________

In [6]:
model_nasnetMobile.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [7]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "NASNetMobile_30 epochs",
      notes = "with more epochs"
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [8]:
%%time
epochs=30
history = model_nasnetMobile.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/30
19/19 [==============================] - ETA: 0s - loss: 2.6719 - accuracy: 0.6115

wandb: Adding directory to artifact (/content/wandb/run-20221019_073622-2q94uifv/files/model-best)... Done. 0.8s


19/19 [==============================] - 246s 9s/step - loss: 2.6719 - accuracy: 0.6115 - val_loss: 0.6564 - val_accuracy: 0.7703
Epoch 2/30
18/19 [===========================>..] - ETA: 0s - loss: 0.5510 - accuracy: 0.7917

wandb: Adding directory to artifact (/content/wandb/run-20221019_073622-2q94uifv/files/model-best)... Done. 0.4s


...

19/19 [==============================] - 88s 5s/step - loss: 0.0052 - accuracy: 0.9983 - val_loss: 0.2295 - val_accuracy: 0.9662
Epoch 27/30
19/19 [==============================] - ETA: 0s - loss: 0.0048 - accuracy: 0.9983

wandb: Adding directory to artifact (/content/wandb/run-20221019_073622-2q94uifv/files/model-best)... Done. 0.3s


19/19 [==============================] - 90s 5s/step - loss: 0.0048 - accuracy: 0.9983 - val_loss: 0.2290 - val_accuracy: 0.9595
Epoch 28/30
19/19 [==============================] - ETA: 0s - loss: 0.0045 - accuracy: 0.9983

wandb: Adding directory to artifact (/content/wandb/run-20221019_073622-2q94uifv/files/model-best)... Done. 0.6s


19/19 [==============================] - 89s 5s/step - loss: 0.0045 - accuracy: 0.9983 - val_loss: 0.2265 - val_accuracy: 0.9595
Epoch 29/30
19/19 [==============================] - 3s 128ms/step - loss: 0.0043 - accuracy: 0.9983 - val_loss: 0.2298 - val_accuracy: 0.9595
Epoch 30/30
19/19 [==============================] - 3s 125ms/step - loss: 0.0042 - accuracy: 0.9983 - val_loss: 0.2269 - val_accuracy: 0.9595
CPU times: user 19min 43s, sys: 36.3 s, total: 20min 19s
Wall time: 27min 17s


In [9]:
wandb.finish()

accuracy,▁▄▅▇▇▇▇███▇██████▇▇███████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▄▆▇▆███▇█▇▇▇█▇▇█▆▇█▇██▇█▇▇▇▇
val_loss,█▇▅▄▃▄▄▄▃▄▄▃▃▃▃▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁
accuracy,0.99831
best_epoch,27
best_val_loss,0.22654
epoch,29
loss,0.0042
val_accuracy,0.95946
